In [1]:
data <- read.csv("Earthquate_Damage.csv")
perf_eval_multi <- function(cm){
  
  # Simple Accuracy
  ACC = sum(diag(cm))/sum(cm)
  
  # Balanced Correction Rate
  BCR = 1
  for (i in 1:dim(cm)[1]){
    BCR = BCR*(cm[i,i]/sum(cm[i,])) 
  }
  
  BCR = BCR^(1/dim(cm)[1])
  
  return(c(ACC, BCR))
}
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
all_train <- rbind(train,val)
val$damage_grade <- as.factor(val$damage_grade)
train$damage_grade <- as.factor(train$damage_grade)
all_train$damage_grade <- as.factor(all_train$damage_grade)
test$damage_grade <- as.factor(test$damage_grade)
x_train <- train[,!(names(train)%in%"damage_grade")]
y_train <- train[,(names(train)%in%"damage_grade")]
x_val <- val[,!(names(val)%in%"damage_grade")]
y_val <- val[,(names(val)%in%"damage_grade")]
x_test <- test[,!(names(test)%in%"damage_grade")]
y_test <- test[,(names(test)%in%"damage_grade")]
x_all_train <- all_train[,!(names(all_train)%in%"damage_grade")]
y_all_train <- all_train[,(names(all_train)%in%"damage_grade")]

In [2]:
install.packages("gbm")
library(gbm)

package 'gbm' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'gbm'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"C:\R\R-3.6.3\library\00LOCK\gbm\libs\x64\gbm.dll를 C:\R\R-3.6.3\library\gbm\libs\x64\gbm.dll로 복사하는데 문제가 발생했습니다: Permission denied"
Warning message:
"restored 'gbm'"



The downloaded binary packages are in
	C:\Users\lmwlm\AppData\Local\Temp\RtmpgbpGGR\downloaded_packages


Loaded gbm 2.1.5



In [3]:
install.packages("caret")
library(caret)

package 'caret' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'caret'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"C:\R\R-3.6.3\library\00LOCK\caret\libs\x64\caret.dll를 C:\R\R-3.6.3\library\caret\libs\x64\caret.dll로 복사하는데 문제가 발생했습니다: Permission denied"
Warning message:
"restored 'caret'"



The downloaded binary packages are in
	C:\Users\lmwlm\AppData\Local\Temp\RtmpgbpGGR\downloaded_packages


Loading required package: lattice

Loading required package: ggplot2



In [19]:
upsample_train <- upSample(subset(x_all_train), y_all_train)

In [20]:
x_upsample_train <- upsample_train[,!(names(upsample_train)%in%"Class")]
y_upsample_train <- upsample_train[,(names(upsample_train)%in%"Class")]

In [21]:
downsample_train <- downSample(subset(x_all_train),y_all_train)

In [22]:
x_downsample_train <- downsample_train[,!(names(downsample_train)%in%"Class")]
y_downsample_train <- downsample_train[,(names(downsample_train)%in%"Class")]

In [10]:
gbm_model <- gbm.fit(x_all_train, y_all_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [23]:
gbm_upsample_model <- gbm.fit(x_upsample_train, y_upsample_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [24]:
gbm_downsample_model <- gbm.fit(x_downsample_train, y_downsample_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [25]:
pred <- predict(gbm_model,newdata=x_test,n.trees=150, type = "response")
dim(pred)<-c(60601,3)
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2277  1126    74
   2  3548 29296  9702
   3    58  4043 10477
[1] 0.6938829 0.6868823


In [26]:
pred <- predict(gbm_upsample_model,newdata=x_test,n.trees=150, type = "response")
dim(pred)<-c(60601,3)
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  4600  6451   889
   2  1123 18449  4879
   3   160  9565 14485
[1] 0.6193627 0.5581931


In [27]:
pred <- predict(gbm_downsample_model,newdata=x_test,n.trees=150, type = "response")
dim(pred)<-c(60601,3)
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  4639  6739   951
   2  1087 18219  4884
   3   157  9507 14418
[1] 0.6151054 0.5536040


In [28]:
pred1 <- predict(gbm_model,newdata=x_test,n.trees=150, type = "response")
dim(pred1)<-c(60601,3)
pred2 <- predict(gbm_upsample_model,newdata=x_test,n.trees=150, type = "response")
dim(pred2)<-c(60601,3)
pred3 <- predict(gbm_downsample_model,newdata=x_test,n.trees=150, type = "response")
dim(pred3)<-c(60601,3)

In [69]:
pred <- pred1*0.9 + pred2*0.05 + pred3*0.05

In [70]:
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2479  1336    88
   2  3334 28661  9242
   3    70  4468 10923
[1] 0.6940975 0.6781548


In [75]:
install.packages("dplyr")
library(dplyr)

also installing the dependencies 'rlang', 'tidyselect', 'vctrs'




package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"C:\R\R-3.6.3\library\00LOCK\rlang\libs\x64\rlang.dll를 C:\R\R-3.6.3\library\rlang\libs\x64\rlang.dll로 복사하는데 문제가 발생했습니다: Permission denied"
Warning message:
"restored 'rlang'"


package 'tidyselect' successfully unpacked and MD5 sums checked
package 'vctrs' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'vctrs'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"C:\R\R-3.6.3\library\00LOCK\vctrs\libs\x64\vctrs.dll를 C:\R\R-3.6.3\library\vctrs\libs\x64\vctrs.dll로 복사하는데 문제가 발생했습니다: Permission denied"
Warning message:
"restored 'vctrs'"


package 'dplyr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'dplyr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"C:\R\R-3.6.3\library\00LOCK\dplyr\libs\x64\dplyr.dll를 C:\R\R-3.6.3\library\dplyr\libs\x64\dplyr.dll로 복사하는데 문제가 발생했습니다: Permission denied"
Warning message:
"restored 'dplyr'"



The downloaded binary packages are in
	C:\Users\lmwlm\AppData\Local\Temp\RtmpgbpGGR\downloaded_packages



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [76]:
class1_tmp <-filter(all_train, damage_grade == c("1"))
class2_tmp <-filter(all_train, damage_grade == c("2"))
class3_tmp <-filter(all_train, damage_grade == c("3"))

In [95]:
nrow(class1_tmp)

[1] 19241

In [96]:
nrow(class2_tmp)

[1] 113794

In [97]:
nrow(class3_tmp)

[1] 66965

In [78]:
set.seed(42)
selec_idx <- sample(1:nrow(class2_tmp), 75000)
class2_tmp <- class2_tmp[selec_idx,]
set.seed(42)
selec_idx <- sample(1:nrow(class3_tmp), 50000)
class3_tmp <- class3_tmp[selec_idx,]

In [79]:
new_train <- rbind(class1_tmp,class2_tmp,class3_tmp)

In [80]:
x_new_train <- new_train[,!(names(new_train)%in%"damage_grade")]
y_new_train <- new_train[,(names(new_train)%in%"damage_grade")]

In [81]:
gbm_new_model <- gbm.fit(x_new_train, y_new_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [82]:
pred <- predict(gbm_new_model,newdata=x_test,n.trees=150, type = "response")
dim(pred)<-c(60601,3)
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2849  1797   146
   2  2948 27388  8507
   3    86  5280 11600
[1] 0.6903681 0.6593266


In [105]:
pred4 <- predict(gbm_new_model,newdata=x_test,n.trees=150, type = "response")
dim(pred4)<-c(60601,3)

In [106]:
pred <- pred1*0.9 + pred4*0.1
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2324  1170    80
   2  3497 29142  9602
   3    62  4153 10571
[1] 0.6936684 0.6845062


In [98]:
class1_tmp <-filter(all_train, damage_grade == c("1"))
class2_tmp <-filter(all_train, damage_grade == c("2"))
class3_tmp <-filter(all_train, damage_grade == c("3"))
set.seed(42)
selec_idx <- sample(1:nrow(class2_tmp), 90000)
class2_tmp <- class2_tmp[selec_idx,]
set.seed(42)
selec_idx <- sample(1:nrow(class3_tmp), 55000)
class3_tmp <- class3_tmp[selec_idx,]

In [99]:
new2_train <- rbind(class1_tmp,class2_tmp,class3_tmp)
x_new2_train <- new2_train[,!(names(new2_train)%in%"damage_grade")]
y_new2_train <- new2_train[,(names(new2_train)%in%"damage_grade")]

In [100]:
gbm_new2_model <- gbm.fit(x_new2_train, y_new2_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [103]:
pred5 <- predict(gbm_new2_model,newdata=x_test,n.trees=150, type = "response")
dim(pred5)<-c(60601,3)

In [104]:
pred <- pred5
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2566  1465   108
   2  3242 28519  9232
   3    75  4481 10913
[1] 0.6930249 0.6725985


In [113]:
pred <- pred1*0.99 + pred5*0.005 +pred4*0.005
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2281  1128    75
   2  3544 29286  9697
   3    58  4051 10481
[1] 0.6938499 0.6867468


In [114]:
class1_tmp <-filter(all_train, damage_grade == c("1"))
class2_tmp <-filter(all_train, damage_grade == c("2"))
class3_tmp <-filter(all_train, damage_grade == c("3"))
set.seed(42)
selec_idx <- sample(1:nrow(class2_tmp), 100000)
class2_tmp <- class2_tmp[selec_idx,]
set.seed(42)
selec_idx <- sample(1:nrow(class3_tmp), 60000)
class3_tmp <- class3_tmp[selec_idx,]
new3_train <- rbind(class1_tmp,class2_tmp,class3_tmp)
x_new3_train <- new3_train[,!(names(new3_train)%in%"damage_grade")]
y_new3_train <- new3_train[,(names(new3_train)%in%"damage_grade")]

In [115]:
gbm_new3_model <- gbm.fit(x_new3_train, y_new3_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [116]:
pred6 <- predict(gbm_new3_model,newdata=x_test,n.trees=150, type = "response")
dim(pred6)<-c(60601,3)

In [117]:
pred <- pred6
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2418  1301    91
   2  3396 28826  9373
   3    69  4338 10789
[1] 0.6936024 0.6784359


In [121]:
pred <- pred1*0.95 + pred6*0.05
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2288  1134    75
   2  3537 29278  9689
   3    58  4053 10489
[1] 0.6939654 0.6866747


In [123]:
class1_tmp <-filter(all_train, damage_grade == c("1"))

set.seed(42)
selec_idx <- sample(1:nrow(class1_tmp), 5000)
class1_tmp <- class1_tmp[selec_idx,]
new4_train <- rbind(class1_tmp,all_train)
x_new4_train <- new4_train[,!(names(new4_train)%in%"damage_grade")]
y_new4_train <- new4_train[,(names(new4_train)%in%"damage_grade")]

In [124]:
gbm_new4_model <- gbm.fit(x_new4_train, y_new4_train, verbose=0,
                 n.trees = 150, shrinkage = 0.1,distribution="multinomial",interaction.depth = 4)

In [125]:
pred7 <- predict(gbm_new4_model,newdata=x_test,n.trees=150, type = "response")
dim(pred7)<-c(60601,3)

In [126]:
pred <- pred7
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2576  1448    92
   2  3240 28804  9634
   3    67  4213 10527
[1] 0.6915232 0.6749703


In [136]:
pred <- pred1*0.999 + pred7*0.0005 + pred6*0.0005
pred <- apply(pred, 1, which.max)
cm <- table(pred,y_test)
print(cm)
print(perf_eval_multi(cm))

    y_test
pred     1     2     3
   1  2277  1126    74
   2  3548 29297  9702
   3    58  4042 10477
[1] 0.6938994 0.6869004
